#### pip install newsapi-python

In [13]:
# -*- coding: utf-8 -*-
from datetime import datetime, date, timedelta
from newspaper import Article
from config import API_KEY, DYNAMODB_ARTICLES_TABLE,BUCKET_NAME
from newsapi  import NewsApiClient
import zipfile
import pandas as pd


ModuleNotFoundError: No module named 'newspaper'

In [11]:



def get_key_word(url):
        key_words = []
        article = Article(url)
        article.download()
        # article.html
        article.parse()
        article.nlp()

        return article.text, article.summary
def get_news_articles(topic,date_from, date_to):
    # Init
    newsapi = NewsApiClient(api_key=API_KEY)
    
    # get everything using topic for query key
    articles= newsapi.get_everything(q=topic,language='en',from_param=date_from,
                                      to=date_to)
    print(articles)
    art=[]
    for i in range(len(articles['articles'])):
        title = articles['articles'][i]['title']
        url = articles['articles'][i]['url']
        blurp = articles['articles'][i]['description']
        imgurl = articles['articles'][i]['urlToImage']
        article_date = articles['articles'][i]['publishedAt']
        article_date = article_date.split('T')[0]
        source = articles['articles'][i]['source']['name'].lower()

        try:
            text, blurp1 = get_key_word(url)
        except:
            text = articles['articles'][i]['content']

        try:
            article_date = datetime.strptime(article_date, "%Y-%m-%d")
            article_date = datetime.strftime(article_date, "%d/%m/%Y")
        except IndexError:
            article_date = date.today().replace(day=1)
            article_date = datetime.strftime(article_date, "%d/%m/%Y")
        art.append(
        {
            'title': title,
            'imgurl': imgurl,
            'date': article_date,
            'blurp': blurp,
            'url': url,
            'text': text,
            'category': topic,
            'source': source
        })
    return art





In [7]:
### set a defined list of topics
# '
topics = ['artificial intelligence', 'cybersecurity', 'blockchain', 'cloud computing','electric vehicles']

In [3]:
dates =[] 
date_from = '2022-11-08'

for day in [7]:
    date_to = datetime.strptime(date_from, '%Y-%m-%d') + timedelta(days=day)
    dates.append(datetime.strftime(date_to,'%Y-%m-%d'))
dates

['2022-11-15']

In [4]:
date_to = '2022-11-15'
# articles2 = get_news_articles('ai',date_from, date_to)

In [12]:
articles =[]
for topic in topics:
    art = get_news_articles(topic,date_from, date_to)
    for a in art:
        articles.append(a)
        
 

NameError: name 'NewsApiClient' is not defined

In [8]:
articles =[]
for topic in topics:
    for d in dates:
        date_to = d
        art = get_news_articles(topic,date_from, date_to)
        for a in art:
            articles.append(a)
        date_from = date_to
 

NameError: name 'get_news_articles' is not defined

In [14]:
data = pd.DataFrame.from_dict(articles, orient='columns')

len(data)

100

In [15]:
data.to_csv('/Users/ruinah/Downloads/project/test_set2.csv')